This Notebook is to present 3 common approaches of doing reference.


1. pipeline default
2. pipeline with specified model and tokenizer
3. using models



#pipeline <br>

By abtracting most of the complex code fabsrom the library, pipeline is the greatest and easiest way to use models for inference. offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.

## 1. pipeline default <br>
same as using model approach, proceed in 3 steps at the backend: <br>
1. tokenization
2. feed token to the model, and run the model
3. return the result

In [ ]:
from transformers import pipeline

inputs = ["I've been waiting for this movie my whole life!", "This movie is fantastic!"]
classifier = pipeline("sentiment-analysis")    # only specify task name = "sentiment-analysis"; will use default model: distilbert
res = classifier(inputs)
print(res)

'''
step 1: tokenization
step 2: feed to the model, and run the model
step 3: return the result
'''

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9926485419273376}]


## 2. pipeline with specified model and tokenizer <br>
tokenizer should be from the model


In [44]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

inputs = ["I've been waiting for this movie my whole life!", "This movie is fantastic!"]
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
res = classifier(inputs)
print(res)

[{'label': 'POSITIVE', 'score': 0.9985682964324951}, {'label': 'POSITIVE', 'score': 0.9998828172683716}]


## 3. Using model, step by step

In [45]:
# it will be useful if we want to fine tune our model

import torch
import torch.nn.functional as F
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs = ["I've been waiting for this movie my whole life!", "This movie is fantastic!"]

batch = tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors="pt")   # tokenization
with torch.no_grad():       # inference, no gradient descent
  output = model(**batch)                          # feed token to the model
  predictions = F.softmax(output.logits, dim=1)    # run the model
  labels = torch.argmax(predictions, dim=1)        # return the result
  print(labels)


tensor([1, 1])


# Miscellaneous

In [13]:
# generating text

from transformers import pipeline

inputs = "In this movie Brokeback Mountain, you will learn love between two men is"
generator = pipeline("text-generation", model="distilgpt2")    # only specify task name = "text-generation"; will use specified model: distilgpt2
res = generator(inputs,
                max_length=50,
                num_return_sequences=2)
print(res)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this movie Brokeback Mountain, you will learn love between two men is an art of love between two.\n\nThe film centers on this very same couple, who find love and love between a very different man.\nThe two are very'}, {'generated_text': 'In this movie Brokeback Mountain, you will learn love between two men is the key to love in a great deal of ways....and with that, some of the same things that matter for you to keep with the rest of your lives.....and'}]


'\nstep 1: tokenization\nstep 2: feed to the model, and run the model\nstep 3: return the result\n'

In [48]:
# multiple classfication

from transformers import pipeline

inputs = "This movie Brokeback Mountain is about love between two men"
generator = pipeline("zero-shot-classification")
res = generator(inputs,
                candidate_labels=["education", "entertainment", "business"])
print(res)


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'This movie Brokeback Mountain is about love between two men', 'labels': ['entertainment', 'business', 'education'], 'scores': [0.9478437900543213, 0.02826233208179474, 0.02389378473162651]}
